In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 300)
import glob
import lightgbm as lgbm
from sklearn.model_selection import KFold

import os
import sys
sys.path.append('..')

from utils import calc_wap, calc_wap2, log_return, realized_volatility, count_unique, calc_mean_importance, calc_model_importance, plot_importance, rmspe, feval_RMSPE
from preprocess import preprocessor

In [19]:
NB = "002"
feature_path = "../output/feature/nb" + NB
model_path = "../output/model/nb" + NB

In [20]:
data_dir = '../input/optiver-realized-volatility-prediction/'

# トレーニングデータ

In [21]:
%%time
train = pd.read_csv(data_dir + 'train.csv')
train_ids = train.stock_id.unique()
df_train = preprocessor(list_stock_ids=train_ids, is_train=True)
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id', 'target']]
df_train = train.merge(df_train, on=['row_id'], how='left')
df_train.head()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  4.9min finished


CPU times: user 6.06 s, sys: 1.77 s, total: 7.82 s
Wall time: 5min


,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.002292,0.006999,0.000008,0.000404,0.002325,0.000388,0.000295,0.117051,0.000852,0.000211,0.257255,0.000176,0.000162,0.053006,-0.000151,0.000126,-0.045557,134.894040,107.260583,40738,323.496689,138.101214,97696,1.003725,0.000693,303.125061,0.001721,-0.000005,0.000210,-0.000361,0.004114,0.000001,0.000503,0.000068,0.000366,0.000277,0.024868,0.000783,0.000181,0.053236,0.000262,0.000178,0.017790,-0.000166,0.000126,-0.011274,141.470588,84.467864,9620,263.941176,116.940077,17948,1.003482,0.000514,68.236749,0.002953,1.131529e-06,0.000251,0.000157,0.004863,0.000002,0.000414,0.000274,0.000372,0.000273,0.051757,0.000822,0.000237,0.114272,0.000223,0.000173,0.030976,-0.000162,0.000131,-0.022548,137.158273,97.898813,19065,294.928058,136.527199,40995,1.003753,0.000487,139.521722,0.003796,0.000001,0.000250,0.000276,0.006087,1.295471e-08,0.000400,0.000003,0.000397,0.000281,0.091997,0.000858,0.000221,0.199058,0.000188,0.000165,0.043697,-0.000147,0.000120,-0.034024,123.586207,103.533216,28672,327.431034,142.761068,75964,1.003832,0.000445,232.888919,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0

# テストデータ

In [22]:
%%time
test = pd.read_csv(data_dir + 'test.csv')
test_ids = test.stock_id.unique()
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


CPU times: user 48.2 ms, sys: 0 ns, total: 48.2 ms
Wall time: 254 ms


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s finished


# ターゲットエンコーディング

In [23]:
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean)

tmp = np.repeat(np.nan, df_train.shape[0])
kf = KFold(n_splits=10, shuffle=True, random_state=55)
for idx_1, idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    
    tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

In [24]:
df_train.head()

,row_id,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id,stock_id_target_enc
0,0-5,0.004136,0.004499,7.613599e-06,0.000260,0.002292,0.006999,0.000008,0.000404,0.002325,0.000388,0.000295,0.117051,0.000852,0.000211,0.257255,0.000176,0.000162,0.053006,-0.000151,0.000126,-0.045557,134.894040,107.260583,40738,323.496689,138.101214,97696,1.003725,0.000693,303.125061,0.001721,-0.000005,0.000210,-0.000361,0.004114,0.000001,0.000503,0.000068,0.000366,0.000277,0.024868,0.000783,0.000181,0.053236,0.000262,0.000178,0.017790,-0.000166,0.000126,-0.011274,141.470588,84.467864,9620,263.941176,116.940077,17948,1.003482,0.000514,68.236749,0.002953,1.131529e-06,0.000251,0.000157,0.004863,0.000002,0.000414,0.000274,0.000372,0.000273,0.051757,0.000822,0.000237,0.114272,0.000223,0.000173,0.030976,-0.000162,0.000131,-0.022548,137.158273,97.898813,19065,294.928058,136.527199,40995,1.003753,0.000487,139.521722,0.003796,0.000001,0.000250,0.000276,0.006087,1.295471e-08,0.000400,0.000003,0.000397,0.000281,0.091997,0.000858,0.000221,0.199058,0.000188,0.000165,0.043697,-0.000147,0.000120,-0.034024,123.586207,103.533216,28672,327.431034,142.761068,75964,1.003832,0.000445,232.888919,0.002006,40.0,3179.0,2.750000,0.001060,14.0,1042.0,

In [25]:
df_test.head()

,stock_id,time_id,row_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_450,log_return_mean_450,log_return_std_450,log_return_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_sum_450,wap_balance_mean_450,wap_balance_std_450,wap_balance_sum_450,price_spread_mean_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_std_450,ask_spread_sum_450,volume_imbalance_mean_450,volume_imbalance_std_450,volume_imbalance_sum_450,total_volume_mean_450,total_volume_std_450,total_volume_sum_450,wap_mean_450,wap_std_450,wap_sum_450,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_150,log_return_mean_150,log_return_std_150,log_return_sum_150,log_return2_realized_volatility_150,log_return2_mean_150,log_return2_std_150,log_return2_sum_150,wap_balance_mean_150,wap_balance_std_150,wap_balance_sum_150,price_spread_mean_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_std_150,ask_spread_sum_150,volume_imbalance_mean_150,volume_imbalance_std_150,volume_imbalance_sum_150,total_volume_mean_150,total_volume_std_150,total_volume_sum_150,wap_mean_150,wap_std_150,wap_sum_150,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc
0,0,4,0-4,0.000294,0.000147,0.000208,0.000294,0.000252,0.000139,0.000159,0.000277,0.000145,0.000022,0.000436,0.000557,0.000028,0.001671,0.000393,0.0,0.00118,-0.000115,0.000113,-0.000344,164.666667,165.711597,494.0,350.666667,65.271229,1052.0,1.000405,0.00017,3.001215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000295,3.0,201.0,3.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,32,0-32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# LightGBM

In [26]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [27]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [28]:
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
  }

In [29]:
kf = KFold(n_splits=5, random_state=55, shuffle=True)
oof = pd.DataFrame()
models = []
scores = 0.0

In [30]:
%%time
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f'Fold: {fold+1}')
    X_train, y_train = X.loc[trn_idx], y.loc[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y.loc[val_idx]
    
    # RMSPEで最適化を行うため? RMSEの損失関数に1/yi^2の重み付けをすればOK
    # https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train, y_train, weight=weights)
    
    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid, y_valid, reference=lgbm_train, weight=weights)
    
    model = lgbm.train(params=params,
                  train_set=lgbm_train,
                  valid_sets=[lgbm_train, lgbm_valid],
                  num_boost_round=5000,         
                  feval=feval_RMSPE,
                  verbose_eval=100,
                  categorical_feature = ['stock_id']                
                 )
    
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)

    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),3)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #keep scores and models
    scores += RMSPE / 5
    models.append(model)
    print("*" * 100)

Fold: 1


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.141231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33923
[LightGBM] [Info] Number of data points in the train set: 343145, number of used features: 134
[LightGBM] [Info] Start training from score 0.001800


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000658412	training's RMSPE: 0.30475	valid_1's rmse: 0.000663155	valid_1's RMSPE: 0.30681
[200]	training's rmse: 0.000532139	training's RMSPE: 0.2463	valid_1's rmse: 0.000541008	valid_1's RMSPE: 0.2503
[300]	training's rmse: 0.000504585	training's RMSPE: 0.23355	valid_1's rmse: 0.00051547	valid_1's RMSPE: 0.23849
[400]	training's rmse: 0.000495601	training's RMSPE: 0.22939	valid_1's rmse: 0.000507304	valid_1's RMSPE: 0.23471
[500]	training's rmse: 0.000490749	training's RMSPE: 0.22714	valid_1's rmse: 0.000503346	valid_1's RMSPE: 0.23288
[600]	training's rmse: 0.000487257	training's RMSPE: 0.22553	valid_1's rmse: 0.000500897	valid_1's RMSPE: 0.23174
[700]	training's rmse: 0.000484182	training's RMSPE: 0.2241	valid_1's rmse: 0.000499009	valid_1's RMSPE: 0.23087
[800]	training's rmse: 0.000481572	training's RMSPE: 0.2229	valid_1's rmse: 0.000497795	valid_1's RMSPE: 0.23031
[900]	training's rmse: 0.0004793

...

Training until validation scores don't improve for 30 rounds
[100]	training's rmse: 0.000657583	training's RMSPE: 0.30472	valid_1's rmse: 0.000665473	valid_1's RMSPE: 0.30643
[200]	training's rmse: 0.000531471	training's RMSPE: 0.24628	valid_1's rmse: 0.000541661	valid_1's RMSPE: 0.24942
[300]	training's rmse: 0.000503793	training's RMSPE: 0.23346	valid_1's rmse: 0.000515231	valid_1's RMSPE: 0.23725
[400]	training's rmse: 0.000494706	training's RMSPE: 0.22924	valid_1's rmse: 0.000507401	valid_1's RMSPE: 0.23365
[500]	training's rmse: 0.000490013	training's RMSPE: 0.22707	valid_1's rmse: 0.000504103	valid_1's RMSPE: 0.23213
[600]	training's rmse: 0.000486476	training's RMSPE: 0.22543	valid_1's rmse: 0.000502221	valid_1's RMSPE: 0.23126
[700]	training's rmse: 0.00048343	training's RMSPE: 0.22402	valid_1's rmse: 0.000500527	valid_1's RMSPE: 0.23048
[800]	training's rmse: 0.000480805	training's RMSPE: 0.2228	valid_1's rmse: 0.0004994	valid_1's RMSPE: 0.22996
[900]	training's rmse: 0.000478

/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096837 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33923
[LightGBM] [Info] Number of data points in the train set: 343146, number of used features: 134
[LightGBM] [Info] Start training from score 0.001801
Training until validation scores don't improve for 30 rounds


/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/yoshikawa/.pyenv/versions/3.7.10/envs/kaggle/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[100]	training's rmse: 0.00065911	training's RMSPE: 0.3049	valid_1's rmse: 0.000657318	valid_1's RMSPE: 0.30478
[200]	training's rmse: 0.000532888	training's RMSPE: 0.24651	valid_1's rmse: 0.000534609	valid_1's RMSPE: 0.24789
[300]	training's rmse: 0.000505098	training's RMSPE: 0.23366	valid_1's rmse: 0.000509185	valid_1's RMSPE: 0.2361
[400]	training's rmse: 0.000495928	training's RMSPE: 0.22941	valid_1's rmse: 0.000502215	valid_1's RMSPE: 0.23287
[500]	training's rmse: 0.000491148	training's RMSPE: 0.2272	valid_1's rmse: 0.00049928	valid_1's RMSPE: 0.2315
[600]	training's rmse: 0.000487655	training's RMSPE: 0.22559	valid_1's rmse: 0.000497542	valid_1's RMSPE: 0.2307
[700]	training's rmse: 0.000484548	training's RMSPE: 0.22415	valid_1's rmse: 0.00049592	valid_1's RMSPE: 0.22995
[800]	training's rmse: 0.000481896	training's RMSPE: 0.22292	valid_1's rmse: 0.000494696	valid_1's RMSPE: 0.22938
[900]	training's rmse: 0.000479606	training's RMSPE: 0.22186	valid_1's rmse: 0.000493759	valid_1

In [31]:
scores

0.23

# Model保存

In [32]:
import pickle

for i, model in enumerate(models):
    pickle.dump(model, open(model_path+"/lgbm"+str(i)+".pkl", 'wb'))

# 特徴量保存

In [33]:
pickle.dump(df_train, open(os.path.join(feature_path, "train.pkl"), 'wb'))
pickle.dump(df_test, open(os.path.join(feature_path, "test.pkl"), 'wb'))

In [37]:
list(X_valid.columns)

['log_return_realized_volatility',
 'log_return_mean',
 'log_return_std',
 'log_return_sum',
 'log_return2_realized_volatility',
 'log_return2_mean',
 'log_return2_std',
 'log_return2_sum',
 'wap_balance_mean',
 'wap_balance_std',
 'wap_balance_sum',
 'price_spread_mean',
 'price_spread_std',
 'price_spread_sum',
 'bid_spread_mean',
 'bid_spread_std',
 'bid_spread_sum',
 'ask_spread_mean',
 'ask_spread_std',
 'ask_spread_sum',
 'volume_imbalance_mean',
 'volume_imbalance_std',
 'volume_imbalance_sum',
 'total_volume_mean',
 'total_volume_std',
 'total_volume_sum',
 'wap_mean',
 'wap_std',
 'wap_sum',
 'log_return_realized_volatility_450',
 'log_return_mean_450',
 'log_return_std_450',
 'log_return_sum_450',
 'log_return2_realized_volatility_450',
 'log_return2_mean_450',
 'log_return2_std_450',
 'log_return2_sum_450',
 'wap_balance_mean_450',
 'wap_balance_std_450',
 'wap_balance_sum_450',
 'price_spread_mean_450',
 'price_spread_std_450',
 'price_spread_sum_450',
 'bid_spread_mean_450